In [1]:
%%python

UsageError: %%python is a cell magic, but the cell body is empty.


In [2]:
sc.stop()

In [3]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import *
import numpy as np
import pandas as pd
sc = SparkContext.getOrCreate()
#
spark = SparkSession(sc)

In [4]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
#sc = SparkContext.getOrCreate()
#spark = SparkSession(sc)

In [5]:
hats = spark.read.csv("hats_data.csv", header="true", inferSchema="true")

In [6]:
pants = spark.read.csv("pants_data.csv", header="true", inferSchema="true")

In [7]:
#pants = pants.select("size","cost","color").filter(col("color") == "blue")

In [8]:
hats.show(5)

+-----+----+-----+-----+
|color|size|price|brand|
+-----+----+-----+-----+
|    0|  31| 1349|    1|
|    2|  30| 2079|    1|
|    1|  30| 1609|    1|
|    2|  32| 2230|    1|
|    2|  31| 1565|    1|
+-----+----+-----+-----+
only showing top 5 rows



STARTING ML

In [9]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree
from pyspark.mllib.util import MLUtils
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.feature import VectorAssembler


In [10]:
data = hats

with assembler

In [11]:
vectorAssembler = VectorAssembler(inputCols = ['color', 'size', 'price'], outputCol = 'features')
#vectorAssembler_label = VectorAssembler(inputCols = ['brand'], outputCol = 'label')

In [12]:
data = vectorAssembler.transform(data)
#data = vectorAssembler_label.transform(data)


In [13]:
data.show(5)

+-----+----+-----+-----+-----------------+
|color|size|price|brand|         features|
+-----+----+-----+-----+-----------------+
|    0|  31| 1349|    1|[0.0,31.0,1349.0]|
|    2|  30| 2079|    1|[2.0,30.0,2079.0]|
|    1|  30| 1609|    1|[1.0,30.0,1609.0]|
|    2|  32| 2230|    1|[2.0,32.0,2230.0]|
|    2|  31| 1565|    1|[2.0,31.0,1565.0]|
+-----+----+-----+-----+-----------------+
only showing top 5 rows



In [14]:
data = data.select('features', 'brand')
data.show(5)

+-----------------+-----+
|         features|brand|
+-----------------+-----+
|[0.0,31.0,1349.0]|    1|
|[2.0,30.0,2079.0]|    1|
|[1.0,30.0,1609.0]|    1|
|[2.0,32.0,2230.0]|    1|
|[2.0,31.0,1565.0]|    1|
+-----------------+-----+
only showing top 5 rows



In [15]:
train_df, test_df  = data.randomSplit([0.7, 0.3])

In [16]:
train_df.show(3)

+-----------------+-----+
|         features|brand|
+-----------------+-----+
|[0.0,28.0,1378.0]|    1|
|[0.0,29.0,2111.0]|    1|
|[0.0,30.0,1553.0]|    1|
+-----------------+-----+
only showing top 3 rows



Classifier in spark

In [17]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [18]:
# Train a RandomForest model.
rf1 = RandomForestClassifier(featuresCol="features", labelCol="brand", numTrees=10)
rf2 = RandomForestClassifier(featuresCol="features", labelCol="brand", numTrees=100)

In [19]:
model1 = rf1.fit(train_df)
model2 = rf2.fit(train_df)


In [20]:
# Make predictions
predictions1 = model2.transform(test_df)
display(predictions1)

DataFrame[features: vector, brand: int, rawPrediction: vector, probability: vector, prediction: double]

In [21]:
predictions1.show(20)

+-----------------+-----+--------------------+--------------------+----------+
|         features|brand|       rawPrediction|         probability|prediction|
+-----------------+-----+--------------------+--------------------+----------+
|[0.0,30.0,1696.0]|    1|[0.0,98.897300944...|[0.0,0.9889730094...|       1.0|
|[0.0,30.0,2717.0]|    1|[0.0,98.476467611...|[0.0,0.9847646761...|       1.0|
|[1.0,28.0,2162.0]|    1|[0.0,99.3125,0.0,...|[0.0,0.993125,0.0...|       1.0|
|[1.0,28.0,2533.0]|    1|[0.0,99.3125,0.0,...|[0.0,0.993125,0.0...|       1.0|
|[1.0,28.0,2682.0]|    1|[0.0,99.3125,0.0,...|[0.0,0.993125,0.0...|       1.0|
|[1.0,28.0,2710.0]|    1|[0.0,99.3125,0.0,...|[0.0,0.993125,0.0...|       1.0|
|[1.0,30.0,2331.0]|    1|[0.0,98.476467611...|[0.0,0.9847646761...|       1.0|
| [2.0,29.0,810.0]|    1|[0.0,95.0,0.0,2.3...|[0.0,0.95,0.0,0.0...|       1.0|
|[2.0,29.0,1273.0]|    1|[0.0,99.0,0.0,0.0...|[0.0,0.99,0.0,0.0...|       1.0|
|[2.0,30.0,2079.0]|    1|[0.0,98.476467611...|[0.0,0

In [22]:
# Make predictions.
predictions = predictions1

# Select example rows to display.
predictions.select("prediction", "brand", "features").show(4)





+----------+-----+-----------------+
|prediction|brand|         features|
+----------+-----+-----------------+
|       1.0|    1|[0.0,30.0,1696.0]|
|       1.0|    1|[0.0,30.0,2717.0]|
|       1.0|    1|[1.0,28.0,2162.0]|
|       1.0|    1|[1.0,28.0,2533.0]|
+----------+-----+-----------------+
only showing top 4 rows



In [23]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="brand", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))
print('accuracy is', accuracy)

Test Error = 0.111111
accuracy is 0.8888888888888888
